In [1]:
import sqlite3
import os
os.makedirs("data/database", exist_ok= True)

In [3]:
conn  = sqlite3.connect("data/database/company.db")
cursor = conn.cursor()

In [7]:
# Create tables
cursor.execute(''' CREATE TABLE IF NOT EXISTS  employees
                (id INTERGER PRIMARY KEY , name TEXT , role TEXT, department TEXT, salary REAL)''')


In [9]:
cursor.execute('''CREATE TABLE IF NOT EXISTS projects
                 (id INTEGER PRIMARY KEY, name TEXT, status TEXT, budget REAL, lead_id INTEGER)''')

In [11]:
# Insert sample data 
employees = [
      (1, 'John Doe', 'Senior Developer', 'Engineering', 95000),
    (2, 'Jane Smith', 'Data Scientist', 'Analytics', 105000),
    (3, 'Mike Johnson', 'Product Manager', 'Product', 110000),
    (4, 'Sarah Williams', 'DevOps Engineer', 'Engineering', 98000)
]

projects = [
    (1, 'RAG Implementation', 'Active', 150000, 1),
    (2, 'Data Pipeline', 'Completed', 80000, 2),
    (3, 'Customer Portal', 'Planning', 200000, 3),
    (4, 'ML Platform', 'Active', 250000, 2)
]

In [13]:
cursor.executemany("insert or replace into employees values (?,?,?,?,?)", employees)
cursor.executemany("insert or replace into employees values (?,?,?,?,?)", projects)


In [15]:
cursor.execute("Select * from employees")

In [17]:
conn.commit()
conn.close()


## Database Content Extraction

In [21]:
from langchain_community.utilities import SQLDatabase
from langchain_community.document_loaders import SQLDatabaseLoader

In [25]:
db = SQLDatabase.from_uri("sqlite:///data/database/company.db")

# Get Database info
print(f"Table : {db.get_usable_table_names()}")
print('\n\n')
print(db.get_table_info())

Table : ['employees', 'projects']




CREATE TABLE employees (
	id INTEGER, 
	name TEXT, 
	role TEXT, 
	department TEXT, 
	salary REAL, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	role	department	salary
1	RAG Implementation	Active	150000	1.0
2	Data Pipeline	Completed	80000	2.0
3	Customer Portal	Planning	200000	3.0
*/


CREATE TABLE projects (
	id INTEGER, 
	name TEXT, 
	status TEXT, 
	budget REAL, 
	lead_id INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from projects table:
id	name	status	budget	lead_id

*/


## Create `SQL` to Documents conversion

In [38]:
from typing import List
from langchain_core.documents import Document

def sql2document(db_path:str)-> List[Document]:
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    documents =[]
    # Create Document for each table
    cursor.execute("select name from sqlite_master where type = 'table';")
    tables = cursor.fetchall()

    for table in tables:
        table_name = table[0]

        # get table schema
        cursor.execute(f"pragma table_info({table_name});")
        columns = cursor.fetchall()
        column_names = [col[1] for col in columns]
        # Get tanble data
        cursor.execute(f"select * from {table_name}")
        rows = cursor.fetchall()

        # Create table overview document
        table_content = f"Table: {table_name}\n"
        table_content += f"Columns: {', '.join(column_names)}\n"
        table_content += f"Total Records: {len(rows)}\n\n"

        # Add sample records
        table_content += "Sample Records: \n"
        for row in rows[:5]:
            record = dict(zip(column_names, row))
            table_content += f"{record}\n"
        
        doc = Document(
            page_content=table_content,
            metadata = {
                'source': db_path,
                'table_name': table_name,
                'num_records': len(rows),
                'data_type': 'sql_table'
            }
        )
        documents.append(doc)

    ## Create relationship documents
    cursor.execute("""
        SELECT e.name, e.role, p.name as project_name, p.status
        FROM employees e
        JOIN projects p ON e.id = p.lead_id
        
        """)
    relationships= cursor.fetchall()
    rel_content = "Employee - Project Relationships:\n\n"
    for rel in relationships:
        rel_content += f"{rel[0]} ({rel[1]}) leads  {rel[2]} -Status: {rel[3]}\n"
    rel_doc = Document(
        page_content=rel_content,
        metadata = {
            'source': db_path,
            'data_type': 'sql_relationships',
            'query': 'employee_project_join'
        }
    )
    documents.append(rel_doc)
    conn.close()
    return documents
    

In [40]:
sql2document("data/database/company.db")

[Document(metadata={'source': 'data/database/company.db', 'table_name': 'employees', 'num_records': 4, 'data_type': 'sql_table'}, page_content="Table: employees\nColumns: id, name, role, department, salary\nTotal Records: 4\n\nSample Records: \n{'id': 1, 'name': 'RAG Implementation', 'role': 'Active', 'department': '150000', 'salary': 1.0}\n{'id': 2, 'name': 'Data Pipeline', 'role': 'Completed', 'department': '80000', 'salary': 2.0}\n{'id': 3, 'name': 'Customer Portal', 'role': 'Planning', 'department': '200000', 'salary': 3.0}\n{'id': 4, 'name': 'ML Platform', 'role': 'Active', 'department': '250000', 'salary': 2.0}\n"),
 Document(metadata={'source': 'data/database/company.db', 'table_name': 'projects', 'num_records': 0, 'data_type': 'sql_table'}, page_content='Table: projects\nColumns: id, name, status, budget, lead_id\nTotal Records: 0\n\nSample Records: \n'),
 Document(metadata={'source': 'data/database/company.db', 'data_type': 'sql_relationships', 'query': 'employee_project_join'